# Grabbing data with cenpy

Cenpy (`sen - pie`) is a package that exposes APIs from the us Census Bureau and makes it easy to pull down and work with Census data in Pandas. First, notice that there are two core modules in the package, `base` and `explorer`, which each do different things. First, let's look at `explorer`. 

In [82]:
import cenpy.explorer as explorer
import pandas

On import, `explorer` request a current version of the APIs available from the Census Bureau. In future, it can also read a `JSON` collection describing the databases from disk, if needed.

Explorer has two functions, `available` and `explain`. `available` will provide a list of the identifiers of all the APIs that `cenpy` knows about. If run with `verbose=True`, `cenpy` will also include the title of the database as a dictionary. It's a good idea to *not* process this directly, and instead use it to explore currently available APIs. 

In [24]:
explorer.available(verbose=True)

{u'1990sf1': u'1990 Decennial: Summary File 1',
 u'2000sf1': u'2000 Decennial: Summary File 1',
 u'2000sf3': u'2000 Decennial: Summary File 3',
 u'2010acs5': u'2010 American Community Survey: 5-Year Estimates',
 u'2010sf1': u'2010 Decennial: Summary File 1',
 u'2011acs5': u'2011 American Community Survey: 5-Year Estimates',
 u'2012acs1': u'2012 American Community Survey: 1-Year Estimates',
 u'2012acs1profile': u'2012 American Community Survey: 1-Year Profile Tables',
 u'2012acs3': u'2012 American Community Survey: 3-Year Estimates',
 u'2012acs3profile': u'2012 American Community Survey: 3-Year Profile Tables',
 u'2012acs5profile': u'2012 American Community Survey: 5-Year Profile Tables',
 u'2012popproj/births': u'Vintage 2012 Population Projections - : Projected Births',
 u'ACSCD1132011': u'2011 American Community Survey 1-Year Profiles for the 113th Congressional Districts',
 u'ACSProfile2013': u'2013 American Community Survey - Data Profiles: 1-Year Data Profile',
 u'ACSProfile3Y2013

The `explain` command provides the title and full description of the datasource. If run in verbose mode, the function returns the full `json` listing of the API. 

In [25]:
explorer.explain('2010acs5')

(u'2010 American Community Survey: 5-Year Estimates',
 u"The American Community Survey (ACS) is a nationwide survey designed to provide communities a fresh look at how they are changing. The ACS replaced the decennial census long form in 2010 and thereafter by collecting long form type information throughout the decade rather than only once every 10 years.  Questionnaires are mailed to a sample of addresses to obtain information about households -- that is, about each person and the housing unit itself.  The American Community Survey produces demographic, social, housing and economic estimates in the form of 1-year, 3-year and 5-year estimates based on population thresholds. The strength of the ACS is in estimating population and housing characteristics. It produces estimates for small areas, including census tracts and population subgroups.  Although the ACS produces population, demographic and housing unit estimates,it is the Census Bureau's Population Estimates Program that produces

To actually connect to a database resource, you create a `Connection`. A `Connection` works like a *very* simplified connection from the `sqlalchemy` world. The `Connection` class has a method, `query` that constructs a query string and requests it from the Census server. This result is then parsed into JSON and returned to the user.  

In [83]:
import cenpy.base as base

In [26]:
conn = base.Connection('2010acs5')

In [27]:
conn

Connection to 2010 American Community Survey: 5-Year Estimates (ID: 2010acs5)

That may have taken longer than you'd've expected. This is because, when the `Connection` constructor is called, it populates the connection object with a bit of metadata that makes it possible to construct queries without referring to the census handbooks. 

For instance, a connection's `variables` represent all available search parameters for a given dataset. 

In [29]:
q.variables.head()

,concept,label,predicateOnly,predicateType
AIANHH,Geographic Summary Level,American Indian Area/Alaska Native Area/Hawaii...,NaN,NaN
AIHHTLI,NaN,American Indian Trust Land/Hawaiian Home Land ...,NaN,NaN
AITS,NaN,American Indian Tribal Subdivision (FIPS),NaN,NaN
AITSCE,NaN,American Indian Tribal Subdivision (Census),NaN,NaN
ANRC,NaN,Alaska Native Regional Corporation (FIPS),NaN,NaN


This dataframe is populated just like the census's table describing the variables on the corresponding [api website](http://api.census.gov/data/2010/sf1/variables.html). Fortunately, this means that you can modify and filter this dataframe just like you can regular pandas dataframes, so working out what the exact codes to use in your query is easy. 

In [30]:
filt = [True if 'B000' in x else False for x in q.variables.index]

In [37]:
q.variables[filt]

,concept,label,predicateOnly,predicateType
B00001_001E,B00001. Unweighted Sample Count of the Popula...,Total,NaN,NaN
B00001_001M,B00001. Unweighted Sample Count of the Popula...,Margin Of Error For!!Total,NaN,NaN
B00002_001E,B00002. Unweighted Sample Housing Units,Total,NaN,NaN
B00002_001M,B00002. Unweighted Sample Housing Units,Margin Of Error For!!Total,NaN,NaN


Likewise, the different levels of geographic scale are determined from the metadata in the overall API listing and recorded.

In [11]:
q.geographies

,geoLevelId,name,optionalWithWCFor,requires
0,010,us,NaN,NaN
1,040,state,NaN,NaN
2,050,county,state,[state]
3,060,county subdivision,NaN,"[state, county]"
4,067,subminor civil subdivision,NaN,"[state, county, county subdivision]"
5,070,place,NaN,"[state, county, county subdivision]"
6,080,tract,NaN,"[state, county, county subdivision, place]"
7,140,tract,county,"[state, county]"
8,150,block group,tract,"[state, county, tract]"
9,160,place,state,[state]


Note that some geographies have a **required** filter to prevent drawing too much data. This will get passed to the `query` method later. 

So, let's just grab the population and margins of error from the ACS. Using the variables table above, we picked out what fields we wanted. In addition, adding the `NAME` field smart-fills the table with the name of the geographic entity being pulled from the census bureau. 

In [38]:
cols = ['B00001_001E', 'B00001_001M', 'B00002_001E', 'B00002_001M', 'NAME'] 

Now the query. The query is constructed just like the API query, and works as follows. 

    cols - list of columns desired from the database
    geo_unit - string denoting the smallest geography to pull (unit of study)
    geo_filter - dictionary containing groupings of geo_units, if required
    
To be specific, a fully query tells the server *what* columns to pull of *what* underlying geography from *what* aggregation units. It's structured using these heterogeneous datatypes so it's easy to change the smallest units quickly, while providing sufficient granularity to change the filters and columns as you go. 

This query below grabs the names, population, and housing estimates from the ACS, as well as their standard errors from census designated places in Arizona. 

In [40]:
data = q.query(cols, geo_unit = 'place:*', geo_filter = {'state':'04'})

Once constructed, the query executes as fast as your internet connection will move. This query has:

In [41]:
data.shape

(451, 7)

Seven columns and 451 rows. So, rather fast. 

For validity and ease of use, we store the last executed query to the object. If you're dodgy about your census API key never being shown in plaintext, never print this property!

In [42]:
q.last_query

u'http://api.census.gov/data/2010/acs5?get=B00001_001E,B00001_001M,B00002_001E,B00002_001M,NAME&for=place:*&in=state:04'

Then, since the results are returned as a pandas dataframe, we can easily process the results. First, it's helpful to know that pandas will not automatically infer numbers from the information returned. So, if you want to convert a numeric column, first mask the data and then cast the column to an integer. 

Using this, I'll select all Census Designated Places in Arizona with populations above 5000.

In [43]:
data[data['B00001_001E'].fillna(0).apply(int) > 5000]

,B00001_001E,B00001_001M,B00002_001E,B00002_001M,NAME,state,place
63,14249,-1,5698,-1,"Chandler city, Arizona",04,12000
146,12256,-1,4335,-1,"Gilbert town, Arizona",04,27400
148,13915,-1,5478,-1,"Glendale city, Arizona",04,27820
224,28472,-1,12137,-1,"Mesa city, Arizona",04,46000
266,9324,-1,3796,-1,"Peoria city, Arizona",04,54050
268,84928,-1,34662,-1,"Phoenix city, Arizona",04,55000
328,12973,-1,6929,-1,"Scottsdale city, Arizona",04,65000
366,7253,-1,3042,-1,"Surprise city, Arizona",04,71510
375,9311,-1,4151,-1,"Tempe city, Arizona",04,73000
394,31499,-1,13967,-1,"Tucson city, Arizona",04,77000


## OK, that's one API, does it work for others?

In [44]:
conn2 = base.Connection('EconCensusEWKS2012')

In [48]:
filt2 = [True if 'Statistics' in x else False for x in conn2.variables['concept']]

In [69]:
conn2.variables[filt2]

,concept,label,predicateOnly,predicateType,required
EMP,Employer Statistics,Number of employees,NaN,string,NaN
EMP_F,Employer Statistics,Flag for Number of employees data suppression ...,NaN,string,NaN
EMP_S,Employer Statistics,Standard error for estimate of number of emplo...,NaN,string,NaN
EMP_S_F,Employer Statistics,Flag for Standard error for estimate of number...,NaN,string,NaN
ESTAB,Employer Statistics,Number of establishments,NaN,string,NaN
ESTAB_F,Employer Statistics,Flag for Number of establishments data suppres...,NaN,string,NaN
NESTAB,Non Employer Statistics,Number of nonemployer establishments,NaN,string,NaN
NESTAB_F,Non Employer Statistics,Flag for Number of nonemployer establishments ...,NaN,string,NaN
NRCPTOT,Non Employer Statistics,"Nonemployer value of business done ($1,000)",NaN,string,NaN
NRCPTOT_F,Non Employer Statistics,Flag for Nonemployer value of business done ($...,NaN,string,NaN


In [47]:
conn2.geographies

,geoLevelId,name,requires
0,001,us,NaN
1,002,state,NaN
2,007,csa,NaN
3,"008,009",metropolitan statistical area/micropolitan sta...,NaN
4,004,consolidated city,[state]
5,003,county,[state]
6,006,place,"[state, county]"
7,005,place,[state]
8,010,metropolitan division,[metropolitan statistical area/micropolitan st...


Here, pay special attention to the `required` column. Also, for this dataset, you need to provide additional parameters. To add however many additional paramters to your query, you add them as keyword arguments at the end. 

In [78]:
d2 = conn2.query(cols=['EMP', 'ESTAB'], geo_unit='place:*', geo_filter={'state':'06'}, OPTAX='*', NAICS2012 = '*')

In [79]:
conn2.last_query

u'http://api.census.gov/data/2012/ewks?get=EMP,ESTAB&NAICS2012=*&OPTAX=*&for=place:*&in=state:06'

In [80]:
d2

,EMP,ESTAB,NAICS2012,OPTAX,state,place
0,0,1,42,10,06,02812
1,0,1,424,10,06,02812
2,0,1,4249,10,06,02812
3,0,1,42493,10,06,02812
4,0,1,424930,10,06,02812
5,0,3,44-45,99,06,02812
6,0,1,445,99,06,02812
7,0,1,4451,99,06,02812
8,0,1,44511,99,06,02812
9,0,1,445110,99,06,02812


And so you get the table of NAICS data for employment and establishments in places in California. 

## ToDo:

1. Provide meaningful feedback from the census errors in the reporting
2. Provide joins with the geodata from the TIGER rest api
3. Provide more convenience functions, like lookups for place and state abbreviations to fips codes?